In [0]:
storageAccountName = 'equitymarketstorage'
storageAccountAccessKey = '32Q+pSDygN/vBS/+2OPfjXJrW9+T4gogsOJY3mEob/GON/Hls27FX1ZaxbBxEjsmA+/+XJn1AXkFj2dUSrOeWA=='
blobContainerName = 'firstcontainer'


if not any(mount.mountPoint == '/mnt/FileStore/MountFolder/' for mount in dbutils.fs.mounts()):
  try:
    dbutils.fs.mount(
    source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
    mount_point = "/mnt/FileStore/MountFolder/",
    extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
  )
  except Exception as e:
    print("already mounted. Try to unmount first")
    
display(dbutils.fs.ls("dbfs:/mnt/FileStore/MountFolder/census_data")) 

path,name,size
dbfs:/mnt/FileStore/MountFolder/census_data/DP02/,DP02/,0
dbfs:/mnt/FileStore/MountFolder/census_data/DP03/,DP03/,0
dbfs:/mnt/FileStore/MountFolder/census_data/DP04/,DP04/,0
dbfs:/mnt/FileStore/MountFolder/census_data/DP05/,DP05/,0
dbfs:/mnt/FileStore/MountFolder/census_data/app.py,app.py,800
dbfs:/mnt/FileStore/MountFolder/census_data/variables/,variables/,0


In [0]:
import requests
import pandas as pd

group_names = ["DP02", "DP03", "DP04", "DP05"]
HOST = "https://api.census.gov/data"

class CensusDataProfiles:
    def __init__(self, profile_name,estimate):
        self.group_name = profile_name
        self.estimate =estimate

    
    def get_dataset(self):
        dataset = "acs/acs{}/profile".format(self.estimate)
        return dataset


    def get_predicates(self):
        predicates = {}
        get_vars = "group({})".format(self.group_name)
        predicates["get"] = get_vars
        predicates["for"] = "county:*"
        predicates["key"] = "e61340309f1ac39404000c3efba3d6921c359b66"
        return predicates


    def download_data_profiles(self):
        for year in range(2009, 2020):
            dataset = self.get_dataset()
            predicates = self.get_predicates()
            base_url = "/".join([HOST, str(year), dataset])
            result = requests.get(base_url, params=predicates)
            df = pd.DataFrame(columns=result.json()[0], data=result.json()[1:])
            df["year"] = year
            df["estimate"]=self.estimate
            df = df.dropna(axis='columns', how='all')
            sparkDF=spark.createDataFrame(df) 
            sparkDF.write.mode("overwrite").parquet('dbfs:/mnt/FileStore/MountFolder/census_data/{}/{}_{}_year_estimate.parquet'.format(self.group_name,year,self.estimate)) 
        

social_data_for_1_year = CensusDataProfiles(group_names[0],1)
social_data_for_1_year.download_data_profiles()

economic_data_for_1_year=CensusDataProfiles(group_names[1],1)
economic_data_for_1_year.download_data_profiles()

housing_data_for_1_year = CensusDataProfiles(group_names[2],1)
housing_data_for_1_year.download_data_profiles()

demographic_data_for_1_year = CensusDataProfiles(group_names[3],1)
demographic_data_for_1_year.download_data_profiles()

social_data_for_5_year = CensusDataProfiles(group_names[0],5)
social_data_for_5_year.download_data_profiles()

economic_data_for_5_year = CensusDataProfiles(group_names[1],5)
economic_data_for_5_year.download_data_profiles()

housing_data_for_5_year = CensusDataProfiles(group_names[2],5)
housing_data_for_5_year.download_data_profiles()

demographic_data_for_5_year = CensusDataProfiles(group_names[3],5)
demographic_data_for_5_year.download_data_profiles()


In [0]:
import json
import pandas as pd

class CensusVariables(CensusDataProfiles):

    def __init__(self,estimate):
        self.estimate = estimate


    def get_dataset(self):
        return super().get_dataset()
    
    
    def get_variables(self):
        for year in range(2010, 2020):
            dataset = self.get_dataset()
            base_url = "/".join([HOST, str(year), dataset,"variables"])
            result = requests.get(base_url)
            df = pd.DataFrame(data=result.json()[1:])
            variables_df = df[df.columns[0:2]]
            sparkDF=spark.createDataFrame(variables_df) 
            sparkDF.write.mode("overwrite").parquet('dbfs:/mnt/FileStore/MountFolder/census_data/variables/{}_{}_year_variables.parquet'.format(year,self.estimate)) 
            
            
variables_1_year = CensusVariables(1)
variables_1_year.get_variables()

variables_5_year = CensusVariables(5)
variables_5_year.get_variables()



In [0]:
display(dbutils.fs.ls("dbfs:/mnt/FileStore/MountFolder/census_data/DP02"))

In [0]:
display(dbutils.fs.ls("dbfs:/mnt/FileStore/MountFolder/census_data/variables"))